# Практическое задание №2

ФИО: Булатов Айдар Салаватович

вуз: МГУ 

факультет: механико-математический

курс: 5

кафедра (если есть): механики композитов

научный руководитель (если есть): Демидович П.Н.

#### Данное задание подготовлено с целью ознакомить слушателей с процедурой подготовки данных для алгоритмов машнинного обучения на примере задачи оценки цены недвижимости. Описание данных можно найти в data_description.txt

### 1. Считывание данных (1 балл)

Считайте данные из файлов train.csv и test.csv в массивы:
trainX (содержит признаки обучающего множества)
trainY (содержит правильные ответа для обучающего множества)
testX (содержит признаки для тестового множества)

Первый столбец содержит порядковый номер объекта, поэтому его рекомендуется сразу удалить

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("train.csv").drop('Id', axis=1)
test = pd.read_csv("test.csv").drop('Id', axis=1)

trainX = train[train.columns[:-1]].values
trainY = train[train.columns[-1]].values
testX = test.values

print(trainX.shape, trainY.shape, testX.shape)

(1460, 79) (1460,) (1459, 79)


In [3]:
print(trainX[0])

[60 'RL' 65.0 8450 'Pave' nan 'Reg' 'Lvl' 'AllPub' 'Inside' 'Gtl'
 'CollgCr' 'Norm' 'Norm' '1Fam' '2Story' 7 5 2003 2003 'Gable' 'CompShg'
 'VinylSd' 'VinylSd' 'BrkFace' 196.0 'Gd' 'TA' 'PConc' 'Gd' 'TA' 'No'
 'GLQ' 706 'Unf' 0 150 856 'GasA' 'Ex' 'Y' 'SBrkr' 856 854 0 1710 1 0 2 1
 3 1 'Gd' 8 'Typ' 0 nan 'Attchd' 2003.0 'RFn' 2 548 'TA' 'TA' 'Y' 0 61 0 0
 0 0 nan nan nan 0 2 2008 'WD' 'Normal']


### 2. Предварительная обработка данных

Как правило "реальные" данные содержат пропущенные значения и прочие нечисловые признаки, поэтому прежде чем запустить методы fit и  predict модели, необходимо сделать все признаки числовыми.

#### 2.1 Обработка пропущенных значений (2 балла)

Изучите раздел документации https://scikit-learn.org/stable/modules/impute.html#impute и параметры классов SimpleImputer и MissingIndicator

In [4]:
from sklearn.impute import MissingIndicator, SimpleImputer

In [5]:
indicator = MissingIndicator()
train_mask_missing_values_only = indicator.fit_transform(trainX)
test_mask_missing_values_only = indicator.fit_transform(testX)
print(np.sum(train_mask_missing_values_only))
print(np.sum(test_mask_missing_values_only))

6965
7000


В данных были обнаружены пропущенные значение. 
При помощи класса MissingIndicator устраните пропущенные значения, использовав самый часто встречаемый элемент.

In [6]:
# ваш код
imp_mf = SimpleImputer(strategy='most_frequent')

trainX_without_missing = imp_mf.fit_transform(trainX)# ваш код
# imp_mf = SimpleImputer(strategy='most_frequent')
testX_without_missing = imp_mf.fit_transform(testX)# ваш код

In [7]:
indicator = MissingIndicator()
train_mask_missing_values_only = indicator.fit_transform(trainX_without_missing)
test_mask_missing_values_only = indicator.fit_transform(testX_without_missing)
print(np.sum(train_mask_missing_values_only))
print(np.sum(test_mask_missing_values_only))

0
0


Пропущенные значения удалены.

In [8]:
print(trainX_without_missing[0])

[60 'RL' 65.0 8450 'Pave' 'Grvl' 'Reg' 'Lvl' 'AllPub' 'Inside' 'Gtl'
 'CollgCr' 'Norm' 'Norm' '1Fam' '2Story' 7 5 2003 2003 'Gable' 'CompShg'
 'VinylSd' 'VinylSd' 'BrkFace' 196.0 'Gd' 'TA' 'PConc' 'Gd' 'TA' 'No'
 'GLQ' 706 'Unf' 0 150 856 'GasA' 'Ex' 'Y' 'SBrkr' 856 854 0 1710 1 0 2 1
 3 1 'Gd' 8 'Typ' 0 'Gd' 'Attchd' 2003.0 'RFn' 2 548 'TA' 'TA' 'Y' 0 61 0
 0 0 0 'Gd' 'MnPrv' 'Shed' 0 2 2008 'WD' 'Normal']


#### 2.2  Обработка категориальных значений (2 балла)

Изучите раздел документации https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features

При помощи класса OrdinalEncoder удалите категориальные признаки.

In [9]:
testX_without_missing[0]

array([20, 'RH', 80.0, 11622, 'Pave', 'Grvl', 'Reg', 'Lvl', 'AllPub',
       'Inside', 'Gtl', 'NAmes', 'Feedr', 'Norm', '1Fam', '1Story', 5, 6,
       1961, 1961, 'Gable', 'CompShg', 'VinylSd', 'VinylSd', 'None', 0.0,
       'TA', 'TA', 'CBlock', 'TA', 'TA', 'No', 'Rec', 468.0, 'LwQ', 144.0,
       270.0, 882.0, 'GasA', 'TA', 'Y', 'SBrkr', 896, 0, 0, 896, 0.0, 0.0,
       1, 0, 2, 1, 'TA', 5, 'Typ', 0, 'Gd', 'Attchd', 1961.0, 'Unf', 1.0,
       730.0, 'TA', 'TA', 'Y', 140, 0, 0, 0, 120, 0, 'Ex', 'MnPrv',
       'Shed', 0, 6, 2010, 'WD', 'Normal'], dtype=object)

In [10]:
#ваш код
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

trainX_without_missing_and_cat = enc.fit_transform(trainX_without_missing)#ваш код
testX_without_missing_and_cat = enc.fit_transform(testX_without_missing)# ваш код

In [11]:
print(trainX_without_missing_and_cat[0])

[  5.   3.  36. 327.   1.   0.   3.   3.   0.   4.   0.   5.   2.   2.
   0.   5.   6.   4. 104.  53.   1.   1.  12.  13.   1. 128.   2.   4.
   2.   2.   3.   3.   2. 361.   5.   0.  73. 179.   1.   0.   1.   4.
 134. 232.   0. 531.   1.   0.   2.   1.   3.   1.   2.   6.   6.   0.
   2.   1.  89.   1.   2. 220.   4.   4.   2.   0.  49.   0.   0.   0.
   0.   2.   2.   2.   0.   1.   2.   8.   4.]


Теперь данные готовы для обучения моделей

Опишите достоинства и недостатки данного метода обработки категориальных признаков

достоинства:
- не так сильно растет размерность в случае большого числа категорий

недостатки:
- добавляем шумовую информацию


### 3. Разделение данных на обучение и валидацию (1 балл)

Разделите обучающее множество на обучение(75%) и валидацию(25%), воспользовавшись функцией train_test_split с random_state=42

In [12]:
# ваш код
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(trainX_without_missing_and_cat,
                                                  trainY,
                                                  train_size=0.75, random_state=42)

In [13]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1095, 79) (365, 79) (1095,) (365,)


### 4. Обучение моделей (4 балла)

Данные готовы для обучения алгоритмов машинного обучения. В качестве базовой можеди возьмём линейную регрессию, меткрика качества - mean_squared_log_error

In [14]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_log_error

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
y_pred[y_pred < 0.0] = 0.0

print(mean_squared_log_error(y_val, y_pred))

0.3817497024775967


Текущую модель можно существенно улучшить. Добейтесь на валидации ошибки меньше 0.03 без каких-либо ограничений на алгоритмы

In [16]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

# from xgboost import XGBRegressor

In [17]:
# model = Lasso()
# model = LogisticRegression()
# model = KNeighborsRegressor()
model = RandomForestRegressor(random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_val)
y_pred[y_pred < 0.0] = 0.0

print(mean_squared_log_error(y_val, y_pred))

0.02059500420904404


#### Лучший результат дала регрессия случайным лесом

### 5. Способы улучшения качества модели

1. Применить другую стратегию обработки пропущенных значений из scikit-learn (1 балл + 1 балл, если получили прирост качества для алгоритма с ошибкой меньше 0.03 )

2. Добавить бинарные признаки отсутствия/присутствия значения для столбцов, где есть пропущенные значения (1 балл)

3. Реализовать самостоятельно одну из стратегий обработки пропущенных значений из семинара (начиная с kNN) (2 балла)

4. Применить стратегию OneHotEncoder  для обработки категориальных признаков (2 балла)

5. Применить стратегию агрегированния данных для обработки категориальных признаков (2 балла)

Реализуйте не менее двух идей из предложенных выше

## 1

#### Попробуем IterativeImputer и KNNImputer. Для категориальных признаков используем SimpleImputer

In [18]:
categorical = []
numerical = []
trainY = train[train.columns[-1]]

for col in train.columns[:-1]:
    for val in train[col]:
        if str(val) == val and not col in categorical:
            categorical.append(col)
            
    if col not in categorical:
        numerical.append(col)
        
train_cat = train[categorical]
train_num = train[numerical]

In [19]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

# num_imp = IterativeImputer()
# num_imp = SimpleImputer(strategy='mean')
num_imp = KNNImputer()
cat_imp = SimpleImputer(strategy='most_frequent')

train_num = num_imp.fit_transform(train_num)
train_cat = cat_imp.fit_transform(train_cat)

In [20]:
trainX_smart_imput = np.hstack([train_num, train_cat])

In [21]:
enc = OrdinalEncoder()
trainX_smart_imput_nocat = enc.fit_transform(trainX_smart_imput)

X_train_smart_imput, X_val_smart_imput, y_train_smart_imput, y_val_smart_imput = train_test_split(trainX_smart_imput_nocat,
                                                  trainY,
                                                  train_size=0.75, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train_smart_imput, y_train_smart_imput)
y_pred = model.predict(X_val_smart_imput)
y_pred[y_pred < 0.0] = 0.0

print(mean_squared_log_error(y_val_smart_imput, y_pred))

0.0204700659025712


In [22]:
from sklearn.model_selection import cross_val_score

In [23]:
print("точность с обработкой пропусков", \
      cross_val_score(model, trainX_smart_imput_nocat, trainY, cv=3).mean())

точность с обработкой пропусков 0.856511144079803


# 2

#### Добавим бинарные признаки отсутствия

In [24]:
train_indicated = train.copy()
test_indicated = test.copy()

ind = MissingIndicator()
indicator_columns = ind.fit_transform(train_indicated)
for i, col in enumerate(indicator_columns.T):
    train_indicated[str(i)] = col

In [25]:
trainX_indicated = train_indicated[train_indicated.columns[:-1]].values

imp_mf = SimpleImputer(strategy='most_frequent')
trainX_without_missing_indicated = imp_mf.fit_transform(trainX_indicated)

enc = OrdinalEncoder()
trainX_without_missing_and_cat_indicated = enc.fit_transform(trainX_without_missing_indicated)

X_train_indicated, X_val_indicated, y_train_indicated, y_val_indicated = train_test_split(trainX_without_missing_and_cat_indicated,
                                                  trainY,
                                                  train_size=0.75, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train_indicated, y_train_indicated)
y_pred = model.predict(X_val_indicated)
y_pred[y_pred < 0.0] = 0.0

print(mean_squared_log_error(y_val_indicated, y_pred))

0.00042119565329856994


In [26]:
print("точность с индикаторами отсутствия: ", \
      cross_val_score(model, trainX_without_missing_and_cat, trainY, cv=3).mean())

точность с индикаторами отсутствия:  0.855113356372537


# 4

#### OneHotEncoder сильно увеличивает размерность и время работы, не уменьшая ошибку. Точность на кросс-валидации меньше

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

trainX_without_missing_and_cat_onehot = enc.fit_transform(trainX_without_missing)

X_train_onehot, X_val_onehot, y_train_onehot, y_val_onehot = train_test_split(trainX_without_missing_and_cat_onehot,
                                                  trainY,
                                                  train_size=0.75, random_state=42)

In [28]:
model = RandomForestRegressor(random_state=42)

model.fit(X_train_onehot, y_train)
y_pred = model.predict(X_val_onehot)
y_pred[y_pred < 0.0] = 0.0

print(mean_squared_log_error(y_val, y_pred))

0.03484946154427984


In [29]:
print("точность с OneHotEncoder", \
      cross_val_score(model, trainX_without_missing_and_cat_onehot, trainY, cv=3).mean())

точность с OneHotEncoder 0.7721093642298994


# 5